## 1. 데이터 로드

In [20]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# 모델링 관련 라이브러리
from sklearn.neighbors import BallTree
from sklearn.model_selection import StratifiedKFold, GroupKFold, StratifiedGroupKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
import xgboost as xgb
from catboost import CatBoostRegressor
from ngboost import NGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 거리 계산 및 공간 분석 라이브러리
from sklearn.neighbors import BallTree

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

In [2]:
# seed 설정
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [3]:
# 파일 경로 설정
file_path = '../data/'

df = pd.read_csv(os.path.join(file_path, 'processed_data.csv'))

In [4]:
# 월별 거래 건수 계산
monthly_transaction_counts = df.groupby('contract_year_month').size().reset_index(name='monthly_transaction_count')

df = df.merge(monthly_transaction_counts, on='contract_year_month', how='left')

# 계약 연도 및 월 추출
df['contract_year'] = df['contract_year_month'].astype(str).str[:4].astype(int)
df['contract_month'] = df['contract_year_month'].astype(str).str[4:6].astype(int)

# 계약 날짜 생성
df['contract_date'] = pd.to_datetime(df['contract_year_month'].astype(str) + df['contract_day'].astype(str), format='%Y%m%d')

# 계약 요일 추출 (0: 월요일, 6: 일요일)
df['contract_weekday'] = df['contract_date'].dt.weekday

In [5]:
df['contract_date'] = df['contract_date'].astype(str)

In [6]:
def reduce_mem_usage(df):
    """
    Iterate through all the columns of a dataframe and modify the data type
    to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2  # Memory usage before optimization
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype
        if str(col_type)=="category":
            continue

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            continue
    end_mem = df.memory_usage().sum() / 1024**2  # Memory usage after optimization
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [7]:
df = reduce_mem_usage(df)

Memory usage of dataframe is 379.64 MB
Memory usage after optimization is: 137.71 MB
Decreased by 63.7%


## **2. 반경 이내 시설물 개수 변수 추가**

### **2.1. 좌표 데이터 준비**

시설물의 위도와 경도 데이터를 사용하여 각 아파트 주변의 시설물 개수를 계산합니다.

In [8]:
# 파일 경로 설정
file_path = '../data/'

# 추가 데이터 로드 (학교, 지하철, 공원 정보)
school_info = pd.read_csv(os.path.join(file_path, 'schoolinfo.csv'))
subway_info = pd.read_csv(os.path.join(file_path, 'subwayInfo.csv'))
park_info = pd.read_csv(os.path.join(file_path, 'parkInfo.csv'))

In [9]:
# 아파트 좌표 추출
apartment_coords = np.radians(df[['latitude', 'longitude']].values)

# 학교 좌표 추출
elementary_coords = np.radians(school_info[school_info['schoolLevel'] == 'elementary'][['latitude', 'longitude']].values)
middle_coords = np.radians(school_info[school_info['schoolLevel'] == 'middle'][['latitude', 'longitude']].values)
high_coords = np.radians(school_info[school_info['schoolLevel'] == 'high'][['latitude', 'longitude']].values)

# 지하철 좌표 추출
subway_coords = np.radians(subway_info[['latitude', 'longitude']].values)

# 공원 좌표 추출
park_coords = np.radians(park_info[['latitude', 'longitude']].values)

### 2.2. BallTree를 사용한 반경 내 시설물 개수 계산 함수

In [10]:
def count_within_radius(apartment_coords, target_coords, radius_km):
    # 지구 반지름 (킬로미터)
    earth_radius = 6371.0
    # 반경을 라디안으로 변환
    radius = radius_km / earth_radius
    # BallTree 생성
    tree = BallTree(target_coords, metric='haversine')
    # 반경 내 시설물 개수 계산
    counts = tree.query_radius(apartment_coords, r=radius, count_only=True)
    return counts

### **2.3. 시설물 개수 계산 및 데이터프레임에 추가**

### **(a) 초등학교, 중학교, 고등학교**

In [11]:
# 반경 설정 (1km, 3km)
school_radii = [1, 3]

for radius in school_radii:
    # 초등학교
    counts_elementary = count_within_radius(apartment_coords, elementary_coords, radius_km=radius)
    df[f'num_elementary_within_{radius}km'] = counts_elementary
    # 중학교
    counts_middle = count_within_radius(apartment_coords, middle_coords, radius_km=radius)
    df[f'num_middle_within_{radius}km'] = counts_middle
    # 고등학교
    counts_high = count_within_radius(apartment_coords, high_coords, radius_km=radius)
    df[f'num_high_within_{radius}km'] = counts_high

### (b) 지하철역

In [12]:
# 반경 설정 (0.5km, 1.5km, 3km)
subway_radii = [0.5, 1.5, 3]

for radius in tqdm(subway_radii):
    counts_subway = count_within_radius(apartment_coords, subway_coords, radius_km=radius)
    df[f'num_subway_within_{radius}km'] = counts_subway

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [01:42<00:00, 34.13s/it]


### (c) 공원

In [13]:
# 반경 설정 (1km, 5km)
park_radii = [1, 5]

for radius in tqdm(park_radii):
    counts_park = count_within_radius(apartment_coords, park_coords, radius_km=radius)
    df[f'num_park_within_{radius}km'] = counts_park

100%|██████████| 2/2 [02:18<00:00, 69.01s/it]


In [14]:
df = reduce_mem_usage(df)

Memory usage of dataframe is 301.48 MB
Memory usage after optimization is: 160.05 MB
Decreased by 46.9%


In [15]:
df = df.drop(['built_year', 'interest_rate'], axis=1)

In [22]:
# df.to_csv("../data/processed_data_2.csv", index=False)

In [16]:
train_df = df.loc[df['_type'] == 'train'].drop('year_month_date', axis=1)
test_df = df.loc[df['_type'] == 'test'].drop('year_month_date', axis=1)

print(f'학습 데이터 크기: {train_df.shape}')
print(f'테스트 데이터 크기: {test_df.shape}')

학습 데이터 크기: (1801228, 34)
테스트 데이터 크기: (150172, 34)


In [17]:
train_df.columns

Index(['index', 'area_m2', 'contract_year_month', 'contract_day',
       'contract_type', 'floor', 'latitude', 'longitude', 'age', 'deposit',
       '_type', 'nearest_subway_distance_km', 'prev_month_interest_rate',
       'nearest_elementary_distance_km', 'nearest_middle_distance_km',
       'nearest_high_distance_km', 'nearest_park_distance_km',
       'nearest_park_area', 'monthly_transaction_count', 'contract_year',
       'contract_month', 'contract_date', 'contract_weekday',
       'num_elementary_within_1km', 'num_middle_within_1km',
       'num_high_within_1km', 'num_elementary_within_3km',
       'num_middle_within_3km', 'num_high_within_3km',
       'num_subway_within_0.5km', 'num_subway_within_1.5km',
       'num_subway_within_3km', 'num_park_within_1km', 'num_park_within_5km'],
      dtype='object')

## **3. 변수 변환**

`floor`와 `age` 등의 변수에 음수나 0이 포함되어 있으므로, Yeo-Johnson 변환을 사용합니다.

### **3.1. 변환할 변수 선택**

In [18]:
# 변환할 수치형 변수 목록
numeric_cols = ['area_m2', 'floor', 'age', 'prev_month_interest_rate', 'nearest_subway_distance_km',
                'nearest_elementary_distance_km', 'nearest_middle_distance_km', 'nearest_high_distance_km',
                'nearest_park_distance_km', 'nearest_park_area',
                'num_elementary_within_1km', 'num_elementary_within_3km',
                'num_middle_within_1km', 'num_middle_within_3km',
                'num_high_within_1km', 'num_high_within_3km',
                'num_subway_within_0.5km', 'num_subway_within_1.5km', 'num_subway_within_3km',
                'num_park_within_1km', 'num_park_within_5km']

### 3.2. Yeo-Johnson 변환 적용

In [19]:
# Yeo-Johnson 변환기 생성
pt = PowerTransformer(method='yeo-johnson')

# 안전하게 변환을 적용하기 위해 열마다 변환 시도
X_train = train_df[numeric_cols].copy()
X_test = test_df[numeric_cols].copy()

successful_cols = []  # 변환에 성공한 컬럼을 추적

# 0 값 또는 음수 값을 작은 양수로 대체하는 함수
def replace_zero_negatives(df, col):
    df[col] = df[col].apply(lambda x: 0.0001 if x <= 0 else x)
    return df

for col in numeric_cols:
    try:
        # 변환 적용 전 0 값 또는 음수 값 처리
        X_train = replace_zero_negatives(X_train, col)
        X_test = replace_zero_negatives(X_test, col)
        
        # 변환 시도
        X_train[col] = pt.fit_transform(X_train[[col]])
        X_test[col] = pt.transform(X_test[[col]])
        successful_cols.append(col)
    except Exception as e:
        print(f"컬럼 '{col}' 변환 중 오류 발생: {e}")

# 변환된 데이터프레임 생성
df_transformed_train = pd.DataFrame(X_train, columns=successful_cols, index=train_df.index)
df_transformed_test = pd.DataFrame(X_test, columns=successful_cols, index=test_df.index)

# 원본 데이터프레임에 변환된 값 업데이트
train_df[successful_cols] = df_transformed_train
test_df[successful_cols] = df_transformed_test

In [24]:
train_df = reduce_mem_usage(train_df)
test_df = reduce_mem_usage(test_df)

Memory usage of dataframe is 374.48 MB
Memory usage after optimization is: 158.04 MB
Decreased by 57.8%
Memory usage of dataframe is 31.22 MB
Memory usage after optimization is: 13.75 MB
Decreased by 56.0%


## 4. 모델링 및 평가

### **4.1. 데이터 준비**

### **(a) 특성과 타겟 변수 분리**

In [33]:
# 타겟 변수
target = 'deposit'

# 사용하지 않을 열
unused_cols = ['index', '_type', 'contract_year_month', 'contract_day', 'contract_date', 'contract_weekday']

# 모델에 사용할 특성 목록
feature_cols = train_df.columns.drop([target] + unused_cols).tolist()

# 훈련 데이터와 테스트 데이터 분리
X = train_df[feature_cols]
y = train_df[target]
X_test = test_df[feature_cols]

### **(b) Stratified Group K-Fold 준비**

`deposit`을 binning하여 stratify하고, `contract_year_month`를 그룹으로 사용합니다.

In [34]:
# 타겟 변수 binning (예: 10구간)
train_df['deposit_bin'] = pd.qcut(y, q=10, labels=False)

# 그룹 변수 설정
groups = train_df['contract_year_month']

# StratifiedGroupKFold 객체 생성
n_splits = 5
skf = StratifiedGroupKFold(n_splits=n_splits, shuffle=True, random_state=42)

### **4.2. 모델 리스트 정의**

사용할 모델들을 딕셔너리로 정의합니다.

In [35]:
models = {
    'LinearRegression': LinearRegression(),
    'Lasso': Lasso(alpha=0.1),
    'Ridge': Ridge(alpha=1.0),
    'ElasticNet': ElasticNet(alpha=0.1, l1_ratio=0.5),
    'DecisionTree': DecisionTreeRegressor(random_state=42),
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
    'LGBM': LGBMRegressor(n_estimators=100, random_state=42),
    'XGBoost': xgb.XGBRegressor(n_estimators=100, random_state=42),
    'CatBoost': CatBoostRegressor(n_estimators=100, verbose=0, random_state=42),
    'NGBoost': NGBRegressor(n_estimators=100, random_state=42)
}

In [36]:
def evaluate_model(model, X, y, groups, cv):
    mae_scores = []
    rmse_scores = []
    fold_scores = []  # 폴드별 결과 저장
    
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, train_df['deposit_bin'], groups), 1):
        X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]

        # 모델 학습
        model.fit(X_train_fold, y_train_fold)
        # 예측
        y_pred = model.predict(X_val_fold)
        # 평가 지표 계산
        mae = mean_absolute_error(y_val_fold, y_pred)
        rmse = np.sqrt(mean_squared_error(y_val_fold, y_pred))
        
        # 폴드별 점수 저장
        mae_scores.append(mae)
        rmse_scores.append(rmse)
        
        # 폴드 결과 출력
        print(f"Fold {fold} - MAE: {mae:.2f}, RMSE: {rmse:.2f}")
        
        # 각 폴드의 결과 저장
        fold_scores.append({
            'Fold': fold,
            'MAE': mae,
            'RMSE': rmse
        })
        
    return np.mean(mae_scores), np.mean(rmse_scores), fold_scores

### 4.4. 모델 학습 및 평가

In [37]:
results = []

for name, model in models.items():
    mae, rmse, fold_scores = evaluate_model(model, X, y, groups, skf)
    
    # 모델의 평균 결과 출력
    print(f"{name} - 평균 MAE: {mae:.2f}, 평균 RMSE: {rmse:.2f}")
    
    # 각 폴드별 결과 저장
    for fold_score in fold_scores:
        results.append({
            'Model': name,
            'Fold': fold_score['Fold'],
            'MAE': fold_score['MAE'],
            'RMSE': fold_score['RMSE']
        })

LinearRegression - MAE: 11230.78, RMSE: 16896.45
